"""
# Formulario Ufficiale delle Copule e loro Densità

Questo documento fornisce le formule matematiche corrette per le funzioni di copula più comuni utilizzate in finanza e le loro densità, con riferimenti alla letteratura standard e implementazioni in Python.

## Notazione
- $C(u, v)$: funzione di copula (bivariata)
- $c(u, v)$: densità della copula
- $F_X(x)$, $F_Y(y)$: funzioni di distribuzione marginali
- $u = F_X(x)$, $v = F_Y(y)$: valori delle marginali (uniformi in [0,1])

## 1. Copula Gaussiana (o Normale)

### Funzione di Copula
$$C(u, v; \rho) = \Phi_{\rho}(\Phi^{-1}(u), \Phi^{-1}(v))$$

dove:
- $\Phi_{\rho}$ è la funzione di distribuzione cumulativa della normale bivariata con correlazione $\rho$
- $\Phi^{-1}$ è l'inversa della funzione di distribuzione cumulativa della normale standard

### Densità della Copula
$$c(u, v; \rho) = \frac{1}{\sqrt{1-\rho^2}} \exp\left(-\frac{\rho^2(x^2 + y^2) - 2\rho xy}{2(1-\rho^2)}\right)$$

dove $x = \Phi^{-1}(u)$ e $y = \Phi^{-1}(v)$.

### Implementazione in Python
```python
def gaussian_copula_pdf(u, v, rho):
    x = stats.norm.ppf(u)
    y = stats.norm.ppf(v)
    
    numerator = np.exp(-(x**2 + y**2 - 2*rho*x*y)/(2*(1-rho**2)))
    denominator = 2*np.pi*np.sqrt(1-rho**2)
    pdf = np.exp(-(x**2 + y**2)/2) / denominator * numerator
    
    return pdf / (stats.norm.pdf(x) * stats.norm.pdf(y))
```

### Log-Densità (per MLE)
```python
def gaussian_copula_logpdf(u, v, rho):
    x = stats.norm.ppf(u)
    y = stats.norm.ppf(v)
    
    term1 = -0.5 * np.log(1 - rho**2)
    term2 = -0.5 * (x**2 + y**2 - 2*rho*x*y)/(1-rho**2) + 0.5 * (x**2 + y**2)
    
    return term1 + term2
```

### Riferimento
- Roger B. Nelsen (2006), "An Introduction to Copulas", Springer, pag. 116-119.

## 2. Copula t-Student

### Funzione di Copula
$$C(u, v; \rho, \nu) = t_{\rho,\nu}(t_\nu^{-1}(u), t_\nu^{-1}(v))$$

dove:
- $t_{\rho,\nu}$ è la funzione di distribuzione cumulativa della t-Student bivariata con correlazione $\rho$ e $\nu$ gradi di libertà
- $t_\nu^{-1}$ è l'inversa della funzione di distribuzione cumulativa della t-Student univariata con $\nu$ gradi di libertà

### Densità della Copula
$$c(u, v; \rho, \nu) = \frac{\Gamma\left(\frac{\nu+2}{2}\right) \Gamma\left(\frac{\nu}{2}\right)^{-1} (1+\frac{x^2 + y^2 - 2\rho xy}{\nu(1-\rho^2)})^{-\frac{\nu+2}{2}}}{\sqrt{1-\rho^2} \cdot \Gamma\left(\frac{\nu+1}{2}\right)^2 \Gamma\left(\frac{\nu}{2}\right)^{-2} (1+\frac{x^2}{\nu})^{-\frac{\nu+1}{2}} (1+\frac{y^2}{\nu})^{-\frac{\nu+1}{2}}}$$

dove $x = t_\nu^{-1}(u)$ e $y = t_\nu^{-1}(v)$.

### Implementazione in Python
```python
def t_copula_pdf(u, v, rho, nu):
    x = stats.t.ppf(u, nu)
    y = stats.t.ppf(v, nu)
    
    num1 = gamma((nu+2)/2) * gamma(nu/2)**-1
    num2 = (1 + (x**2 + y**2 - 2*rho*x*y)/(nu*(1-rho**2)))**(-((nu+2)/2))
    
    den1 = np.sqrt(1-rho**2)
    den2 = gamma((nu+1)/2)**2 * gamma(nu/2)**-2
    den3 = (1 + x**2/nu)**(-(nu+1)/2) * (1 + y**2/nu)**(-(nu+1)/2)
    
    return (num1 * num2) / (den1 * den2 * den3)
```

### Log-Densità (per MLE, forma semplificata)
```python
def t_copula_logpdf(u, v, rho, nu):
    x = stats.t.ppf(u, nu)
    y = stats.t.ppf(v, nu)
    
    # Calcolo della densità della t bivariata
    term1 = -0.5 * np.log(1 - rho**2)
    term2 = -(nu + 2)/2 * np.log(1 + (x**2 + y**2 - 2*rho*x*y)/(nu*(1-rho**2)))
    
    # Sottrarre le densità marginali
    term3 = -stats.t.logpdf(x, nu) - stats.t.logpdf(y, nu)
    
    return term1 + term2 - term3
```

### Riferimento
- Demarta, S. and McNeil, A. J. (2005), "The t Copula and Related Copulas", International Statistical Review, 73(1), 111-129.

## 3. Copula di Clayton

### Funzione di Copula
$$C(u, v; \theta) = \max\{(u^{-\theta} + v^{-\theta} - 1)^{-1/\theta}, 0\}$$

dove $\theta > 0$ controlla la dipendenza (maggiore θ = maggiore dipendenza positiva).

### Densità della Copula
$$c(u, v; \theta) = (1+\theta) (u \cdot v)^{-(1+\theta)} (u^{-\theta} + v^{-\theta} - 1)^{-2-1/\theta}$$

### Implementazione in Python
```python
def clayton_copula_pdf(u, v, theta):
    if theta <= 0:
        raise ValueError("theta deve essere > 0 per la copula di Clayton")
    
    term1 = (1 + theta)
    term2 = (u * v)**-(1 + theta)
    term3 = (u**-theta + v**-theta - 1)**(-2 - 1/theta)
    
    return term1 * term2 * term3
```

### Log-Densità (per MLE)
```python
def clayton_copula_logpdf(u, v, theta):
    if theta <= 0:
        return -np.inf
    
    term1 = np.log(1 + theta)
    term2 = -(1 + theta) * np.log(u * v)
    term3 = -(2 + 1/theta) * np.log(u**-theta + v**-theta - 1)
    
    return term1 + term2 + term3
```

### Riferimento
- Clayton, D. G. (1978), "A model for association in bivariate life tables and its application in epidemiological studies of familial tendency in chronic disease incidence", Biometrika, 65, 141-151.

## 4. Copula di Gumbel

### Funzione di Copula
$$C(u, v; \theta) = \exp\left\{-\left[(-\ln u)^\theta + (-\ln v)^\theta\right]^{1/\theta}\right\}$$

dove $\theta \geq 1$ controlla la dipendenza (θ = 1: indipendenza, θ → ∞: dipendenza perfetta).

### Densità della Copula
$$c(u, v; \theta) = \frac{C(u, v; \theta) \cdot \left[(-\ln u)^\theta + (-\ln v)^\theta\right]^{2/\theta-2} \cdot (\ln u \cdot \ln v)^{\theta-1}}{u \cdot v} \cdot \left(\theta-1 + \left[(-\ln u)^\theta + (-\ln v)^\theta\right]^{1/\theta}\right)$$

### Implementazione in Python
```python
def gumbel_copula_pdf(u, v, theta):
    if theta < 1:
        raise ValueError("theta deve essere ≥ 1 per la copula di Gumbel")
    
    log_u = -np.log(u)
    log_v = -np.log(v)
    sum_term = (log_u**theta + log_v**theta)**(1/theta)
    
    C = np.exp(-sum_term)  # La funzione di copula stessa
    
    term1 = C * (log_u**theta + log_v**theta)**(2/theta - 2)
    term2 = (log_u * log_v)**(theta-1)
    term3 = (theta-1 + sum_term)
    term4 = 1/(u*v)
    
    return term1 * term2 * term3 * term4
```

### Log-Densità (per MLE, forma semplificata)
```python
def gumbel_copula_logpdf(u, v, theta):
    if theta < 1:
        return -np.inf
    
    log_u = -np.log(u)
    log_v = -np.log(v)
    w = (log_u**theta + log_v**theta)**(1/theta)
    
    term1 = -w  # log(C)
    term2 = np.log(w * (log_u*log_v)**(theta-1))
    term3 = np.log(theta-1 + w) - np.log(u*v)
    
    return term1 + term2 + term3
```

### Riferimento
- Gumbel, E. J. (1960), "Bivariate exponential distributions", Journal of the American Statistical Association, 55, 698-707.

## 5. Copula di Frank

### Funzione di Copula
$$C(u, v; \theta) = -\frac{1}{\theta} \ln\left(1 + \frac{(e^{-\theta u} - 1)(e^{-\theta v} - 1)}{e^{-\theta} - 1}\right)$$

dove $\theta \neq 0$ controlla la dipendenza (θ > 0: dipendenza positiva, θ < 0: dipendenza negativa).

### Densità della Copula
$$c(u, v; \theta) = \frac{\theta (1-e^{-\theta}) e^{-\theta(u+v)}}{(1-e^{-\theta} - (1-e^{-\theta u})(1-e^{-\theta v}))^2}$$

### Implementazione in Python
```python
def frank_copula_pdf(u, v, theta):
    if theta == 0:
        return 1.0  # Caso di indipendenza
    
    num = theta * (1 - np.exp(-theta)) * np.exp(-theta * (u + v))
    den = (1 - np.exp(-theta) - (1 - np.exp(-theta * u)) * (1 - np.exp(-theta * v)))**2
    
    return num / den
```

### Log-Densità (per MLE)
```python
def frank_copula_logpdf(u, v, theta):
    if abs(theta) < 1e-10:
        return 0.0  # Caso di indipendenza
    
    term1 = np.log(abs(theta)) + np.log(1 - np.exp(-abs(theta)))
    term2 = -theta * (u + v)
    term3 = -2 * np.log(1 - np.exp(-theta) - (1 - np.exp(-theta * u)) * (1 - np.exp(-theta * v)))
    
    return term1 + term2 + term3
```

### Riferimento
- Frank, M. J. (1979), "On the simultaneous associativity of F(x, y) and x + y - F(x, y)", Aequationes Mathematicae, 19, 194-226.

## 6. Relazioni tra parametri delle copule e misure di dipendenza

### Tau di Kendall

Le formule che legano il tau di Kendall ai parametri delle diverse copule:

1. **Copula Gaussiana**:
   $$\tau = \frac{2}{\pi} \arcsin(\rho)$$

   Inversione: $$\rho = \sin(\frac{\pi\tau}{2})$$

2. **Copula t-Student**:
   $$\tau = \frac{2}{\pi} \arcsin(\rho)$$
   
   Stessa relazione della Gaussiana, indipendente dai gradi di libertà ν.

3. **Copula di Clayton**:
   $$\tau = \frac{\theta}{\theta+2}$$
   
   Inversione: $$\theta = \frac{2\tau}{1-\tau}$$

4. **Copula di Gumbel**:
   $$\tau = 1 - \frac{1}{\theta}$$
   
   Inversione: $$\theta = \frac{1}{1-\tau}$$

5. **Copula di Frank**:
   $$\tau = 1 - \frac{4}{\theta}(1 - D_1(\theta))$$
   
   dove $D_1(\theta)$ è la funzione di Debye:
   $$D_1(\theta) = \frac{1}{\theta}\int_0^\theta \frac{t}{e^t-1}dt$$
   
   Non esiste una formula di inversione semplice.

## 7. Scelta della Copula più Adatta

Per determinare quale copula rappresenti meglio la dipendenza nei dati, si possono utilizzare criteri di informazione come AIC o BIC:

- **AIC** (Akaike Information Criterion): $$\text{AIC} = -2 \ln(L) + 2k$$
- **BIC** (Bayesian Information Criterion): $$\text{BIC} = -2 \ln(L) + k \ln(n)$$

dove:
- $L$ è la massima verosimiglianza del modello
- $k$ è il numero di parametri
- $n$ è il numero di osservazioni

Il modello con AIC e BIC più bassi è generalmente preferibile.

## Riferimenti Generali

1. Nelsen, R. B. (2006). An Introduction to Copulas (2nd ed.). New York: Springer.
2. Joe, H. (2014). Dependence Modeling with Copulas. Chapman and Hall/CRC.
3. Cherubini, U., Luciano, E., & Vecchiato, W. (2004). Copula Methods in Finance. Wiley Finance.
4. McNeil, A. J., Frey, R., & Embrechts, P. (2015). Quantitative Risk Management: Concepts, Techniques and Tools. Princeton University Press.
"""